<a href="https://colab.research.google.com/github/trishla18/NanoGPT/blob/main/PDF_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import nltk
import re , math
import gensim
from gensim.parsing.preprocessing import remove_stopwords
from gensim import corpora
import re
!pip install tika
!pip install PyPDF2
import PyPDF2
from tika import parser

  Preparing metadata (setup.py) ... done
  Created wheel for tika: filename=tika-2.6.0-py3-none-any.whl size=32621 sha256=39b084a7a67f7135f71a7fab91733270f11a8893dd15575fd51f4fe20be09a7e
  Stored in directory: /root/.cache/pip/wheels/5f/71/c7/b757709531121b1700cffda5b6b0d4aad095fb507ec84316d0
Successfully built tika
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [ ]:
ppt_list=["/content/ppts/QT-EMIEMC.pdf","/content/ppts/QT-Environment.pdf","/content/ppts/QT-Introduction.pdf","/content/ppts/QT-Powersupply.pdf"]

In [ ]:
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!pip install num2words

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=385a79bf244dd289e728d08b55b65a37cba5f5d34fdb88a6936f62565379ca0e
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from collections import Counter
import num2words
num2words.num2words(345)

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle
import re
import math

In [ ]:
ppt_list

['/content/ppts/QT-EMIEMC.pdf',
 '/content/ppts/QT-Environment.pdf',
 '/content/ppts/QT-Introduction.pdf',
 '/content/ppts/QT-Powersupply.pdf']

In [ ]:
dataset=[]
for i in ppt_list:
    dataset.append( (str(i) ,i.split("/")[-1].split(".")[0]))
dataset

[('/content/ppts/QT-EMIEMC.pdf', 'QT-EMIEMC'),
 ('/content/ppts/QT-Environment.pdf', 'QT-Environment'),
 ('/content/ppts/QT-Introduction.pdf', 'QT-Introduction'),
 ('/content/ppts/QT-Powersupply.pdf', 'QT-Powersupply')]

In [ ]:
N=len(dataset)

In [ ]:
def print_doc(id):
    print(dataset[id])
    file = open(dataset[id][0], 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    print(text)

In [ ]:
def convert_lower_case(data):
    return np.char.lower(data)

In [ ]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [ ]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

In [ ]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

In [ ]:
def stemming(data):
    stemmer= PorterStemmer()

    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [ ]:
def convert_numbers(data):
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [ ]:
def preprocess(data):
    data = convert_lower_case(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_apostrophe(data)
    data = remove_stop_words(data)
    data = convert_numbers(data)
    data = stemming(data)
    data = remove_punctuation(data)
    data = convert_numbers(data)
    data = stemming(data) #needed again as we need to stem the words
    data = remove_punctuation(data) #needed again as num2word is giving few hypens and commas fourty-one
    data = remove_stop_words(data) #needed again as num2word is giving stop words 101 - one hundred and one
    return data

In [ ]:

processed_text = []
processed_title = []

for i in dataset[:N]:
  file_path, title = i
  if file_path.endswith(".pdf"):
      try:
        with open(file_path, 'rb') as pdf_file:
          pdf_reader = PyPDF2.PdfReader(pdf_file)
          text = ""
          for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
      except:
        text = " (Text extraction failed for this PDF)"
    # file = open(i[0], 'r', encoding="utf8", errors='ignore')
    # text = file.read().strip()
    # file.close()

  processed_text.append(word_tokenize(str(preprocess(text))))
  processed_title.append(word_tokenize(str(preprocess(i[1]))))

In [ ]:
processed_text

[['slide',
  '1qualif',
  'test',
  'avion',
  'lrusauthor',
  'page',
  'divi',
  'page',
  'slide',
  'two',
  'element',
  'electro',
  'magnet',
  'coupl',
  'radiat',
  'conduct',
  'emi',
  'coupl',
  'mechan',
  'emiss',
  'suscept',
  'emi',
  'emcqualif',
  'test',
  'emi',
  'emc',
  'author',
  'page',
  'divi',
  'page',
  'slide',
  'purpo',
  '\uf0d8effect',
  'radiat',
  'conduct',
  'inevit',
  'electron',
  'equip',
  '\uf0d8they',
  'limit',
  'follow',
  'proper',
  'emi',
  'emc',
  'design',
  '\uf0d8limit',
  'specifi',
  'standard',
  '•defin',
  'frequenc',
  'rang',
  'limit',
  'unint',
  'radiat',
  '•defin',
  'interf',
  'frequenc',
  'rang',
  'level',
  'equip',
  'shall',
  'perform',
  'emi',
  'emc',
  'standard',
  '\uf0d8cispr',
  'iec',
  'commerci',
  'ism',
  'equip',
  'hou',
  'hold',
  'applianc',
  'etc',
  '\uf0d8iso',
  'sae',
  'automot',
  '\uf0d8mil',
  'std',
  '461',
  'militari',
  'equip',
  '\uf0d8do',
  '164',
  'civil',
  'aviat',


In [ ]:
processed_title

[['qt', 'emiemc'],
 ['qt', 'environ'],
 ['qt', 'introduct'],
 ['qt', 'powersuppli']]

In [ ]:
DF = {}
for i in range(len(processed_text)):
    tokens = processed_text[i]
    for w in tokens:
        try:
            DF[w].add(i)
        except:
            DF[w] = {i}

In [ ]:
total_vocab_size = len(DF)
total_vocab_size

1124

In [ ]:
total_vocab=[x for x in DF]

In [ ]:
total_vocab[:20]

['slide',
 '1qualif',
 'test',
 'avion',
 'lrusauthor',
 'page',
 'divi',
 'two',
 'element',
 'electro',
 'magnet',
 'coupl',
 'radiat',
 'conduct',
 'emi',
 'mechan',
 'emiss',
 'suscept',
 'emcqualif',
 'emc']

In [ ]:
for i in DF:
    DF[i] = len(DF[i])
DF

{'slide': 4,
 '1qualif': 2,
 'test': 4,
 'avion': 3,
 'lrusauthor': 2,
 'page': 4,
 'divi': 4,
 'two': 1,
 'element': 1,
 'electro': 1,
 'magnet': 1,
 'coupl': 2,
 'radiat': 1,
 'conduct': 2,
 'emi': 2,
 'mechan': 2,
 'emiss': 1,
 'suscept': 1,
 'emcqualif': 1,
 'emc': 2,
 'author': 4,
 'purpo': 3,
 '\uf0d8effect': 1,
 'inevit': 1,
 'electron': 2,
 'equip': 3,
 '\uf0d8they': 1,
 'limit': 2,
 'follow': 3,
 'proper': 1,
 'design': 3,
 '\uf0d8limit': 1,
 'specifi': 4,
 'standard': 4,
 '•defin': 1,
 'frequenc': 3,
 'rang': 2,
 'unint': 1,
 'interf': 1,
 'level': 3,
 'shall': 3,
 'perform': 4,
 '\uf0d8cispr': 1,
 'iec': 1,
 'commerci': 1,
 'ism': 1,
 'hou': 1,
 'hold': 1,
 'applianc': 1,
 'etc': 2,
 '\uf0d8iso': 1,
 'sae': 1,
 'automot': 2,
 '\uf0d8mil': 1,
 'std': 3,
 '461': 1,
 'militari': 2,
 '\uf0d8do': 1,
 '164': 1,
 'civil': 1,
 'aviat': 1,
 'qualif': 4,
 'mil': 3,
 'specif': 4,
 'requir': 4,
 'ce': 1,
 'rs': 1,
 'cs': 1,
 '5qualif': 1,
 'applic': 4,
 'matrixauthor': 1,
 '6emi': 1,
 '

In [ ]:
def doc_freq(word):
    c = 0
    try:
        c = DF[word]
    except:
        pass
    return c

In [ ]:
doc = 0

tf_idf = {}

for i in range(N):

    tokens = processed_text[i]

    counter = Counter(tokens + processed_title[i])
    words_count = len(tokens + processed_title[i])

    for token in np.unique(tokens):

        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1))

        tf_idf[doc, token] = tf*idf

    doc += 1

In [ ]:
doc = 0

tf_idf_title = {}

for i in range(N):

    tokens = processed_title[i]
    counter = Counter(tokens + processed_text[i])
    words_count = len(tokens + processed_text[i])

    for token in np.unique(tokens):

        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = np.log((N+1)/(df+1)) #numerator is added 1 to avoid negative values

        tf_idf_title[doc, token] = tf*idf

    doc += 1

In [ ]:
tf_idf[(1, 'environ')]

0.013245262500414625

In [ ]:
alpha = 0.3

In [ ]:
for i in tf_idf:
    tf_idf[i] *= alpha
for i in tf_idf_title:
    tf_idf[i] = tf_idf_title[i]

In [ ]:
def cosine_sim(a, b):
    cos_sim = np.dot(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    return cos_sim

In [ ]:
D = np.zeros((N, total_vocab_size))
for i in tf_idf:
    try:
        ind = total_vocab.index(i[1])
        D[i[0]][ind] = tf_idf[i]
    except:
        pass

In [ ]:
def gen_vector(tokens):

    Q = np.zeros((len(total_vocab)))

    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}

    for token in np.unique(tokens):

        tf = counter[token]/words_count
        df = doc_freq(token)
        idf = math.log((N+1)/(df+1))

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf*idf
        except:
            pass
    return Q

In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("\nQuery:", query)
    print("")
    print(tokens)

    d_cosines = []

    query_vector = gen_vector(tokens)

    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]

    print("")

    print(out)

    for i in out:
       print(i, dataset[i][1])


Q = cosine_similarity(10,"Mention the Military standard that is used for Environmental testing ")

Cosine Similarity

Query: Mention the Military standard that is used for Environmental testing 

['mention', 'militari', 'standard', 'use', 'environ', 'test']

[1 2 0 3]
1 QT-Environment
2 QT-Introduction
0 QT-EMIEMC
3 QT-Powersupply


In [ ]:
def cosine_similarity(k, query):
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    tokens = word_tokenize(str(preprocessed_query))

    print("\nQuery:", query)
    print("")
    print(tokens)

    d_cosines = []

    query_vector = gen_vector(tokens)

    for d in D:
        d_cosines.append(cosine_sim(query_vector, d))

    out = np.array(d_cosines).argsort()[-k:][::-1]

    print("")

    print(out)

    for i in out:
       print(i, dataset[i][1])
    answer_candidates = []
    for i in out:
      title, doc_text = dataset[i]  # Access title and document text from the dataset

    # Implement your logic to identify answerable documents and extract answer snippets (text_snippet)
    # This is a placeholder for now
    text_snippet = " (Answer extraction not implemented yet)"  # Placeholder

    answer_candidates.append({
        'title': title,
        'text_snippet': text_snippet
    })
    print(f"- {title}")  # Print the title with formatting

    return answer_candidates
Q = cosine_similarity(10,"What are the exteme temperature range in temperature test")

Cosine Similarity

Query: What are the exteme temperature range in temperature test

['extem', 'temperatur', 'rang', 'temperatur', 'test']

[1 0 3 2]
1 QT-Environment
0 QT-EMIEMC
3 QT-Powersupply
2 QT-Introduction
- /content/ppts/QT-Introduction.pdf


In [ ]:
Q = cosine_similarity(10,"Rapid Decompression test is covered under whch test")

NameError: name 'cosine_similarity' is not defined